In [1]:
import requests
from bs4 import *
import pandas as pd

In [2]:
class MonthlyTransation:
    def __init__(self, exchangeName, year, month):
        self.exchangeName = exchangeName
        self.year = year
        self.month = month
        self.turnover = 0
        self.volume = 0
        self.cumulativeTurnover = 0
        self.cumulativeVolume = 0
    
    def setTurnoverVolume(self, turnover, volume):
        self.turnover = turnover
        self.volume = volume
        
    def setCumTurnoverVolume(self, cumulativeTurnover, cumulativeVolume):
        self.cumulativeTurnover = cumulativeTurnover
        self.cumulativeVolume = cumulativeVolume

In [3]:
def getMonthlyTransactionInfo2(paragraph, index, year, month):    
    content = paragraph.text
    
    content = content.replace(',', '')
    content = content.replace('。', '')
    content = content.replace('，', '')
    
    exchangeName = getExchangeName(content)
    
    if (exchangeName == ''):
        exchangeName = '全国期货市场'

    transaction = MonthlyTransation(exchangeName, year, month)
#     print('{}/{}'.format(transaction.year, transaction.month))
    transaction.setTurnoverVolume(parseTurnover(content), parseVolume(content))
    
    if (month == 1):
        transaction.setCumTurnoverVolume(parseTurnover(content), parseVolume(content))
        
        return transaction
        
    transaction.setCumTurnoverVolume(parseCumTurnover(content), parseCumVolume(content))
    
    return transaction


def parseVolume(str):
    begin = str.find('成交量为')
    end = str.find('手', begin)
    
    return int(str[begin+4:end])

def parseTurnover(str):
    begin = str.find('成交额为')
    end = str.find('亿', begin)
    
    return float(str[begin+4:end])

def parseCumVolume(str):
    begin = str.find('累计成交量为')
    end = str.find('手', begin)
    
    return int(str[begin+6:end])

def parseCumTurnover(str):
    begin = str.find('累计成交额为')
    end = str.find('亿', begin)
    
    return float(str[begin+6:end])

def getExchangeName(str):
    end = str.find('交易所')
    
    # handle 交易中心
    if (end == -1):
        end = str.find('交易中心')
        
        if (end == -1):
            return ''
        
        return str[:end+4].replace(u'\u3000', '').strip()
    
    return str[:end+3].replace(u'\u3000', '').strip()

def getDateTime(str):
    year = int(str[:4])
    month = int(str[str.find('年')+1:str.find('月')])
    
    return year, month

In [4]:
def crawlMonthlyTransation(url):
    transaction_dict = {}
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('UTF-8', errors='ignore'), 'html.parser')
    
    title_text = soup.find('div', class_='job-tit').text 
    year, month = getDateTime(title_text)
    
    div = soup.find('div', class_='content')
    all_p = div.findAll('p')
    
    if (len(all_p) == 0):
        all_p = div.findAll('span')
    
    if (all_p[0].text.strip() == ''):
        all_p = all_p[1:]

    for i, p in enumerate(all_p):
        try:
            info = getMonthlyTransactionInfo2(p, i, year, month)

#             transation_dict[info.exchangeName] = info
            
            if (i == 0):
                print('{}/{}'.format(info.year, info.month))

            print(
                info.exchangeName, 
                info.volume,
                info.turnover, 
                info.cumulativeVolume,
                info.cumulativeTurnover)
            
            transaction_dict[info.exchangeName] = info
            
        except:
            continue

    return transaction_dict


In [5]:
url = 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/201608/t20160801_8541.html'

dic = crawlMonthlyTransation(url)

dic


2016/7
上海期货交易所 160202862 87359.17 1103866283 512327.46
郑州商品交易所 99202298 36881.93 557716294 183215.79
大连商品交易所 137495625 52727.17 1016332004 381637.02
中国金融期货交易所 1337068 13673.92 10882758 106874.99


{'上海期货交易所': <__main__.MonthlyTransation at 0x7f969c12f3a0>,
 '郑州商品交易所': <__main__.MonthlyTransation at 0x7f969c23fdf0>,
 '大连商品交易所': <__main__.MonthlyTransation at 0x7f969c24a220>,
 '中国金融期货交易所': <__main__.MonthlyTransation at 0x7f969c24a610>}

In [6]:
from urllib.parse import urljoin

def getCurrentPageUrls(url):
    urls = []

    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('UTF-8', errors='ignore'), 'html.parser')
    div = soup.find('div', class_='mod-news-3')
    ul = div.find('ul')
    ls = ul.find_all('li')
    
    for li in ls:
        link = li.find('a', href=True)
        urls.append(urljoin(url, link['href']))
        
    return urls

def getAllPageUrls(url):
    baseUrl = 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata'
    urls = []

    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('UTF-8', errors='ignore'), 'html5lib')
    div = soup.find('div', class_='page moreBox')
    
    print(div)
    allA = div.find_all('a', href=True)
    
    
    for a in allA:
        link = a['href']
        urls.append(urljoin(baseUrl, link['href']))
        
    return urls

In [7]:

# getAllPageUrls('http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index.html')

# list all page urls
baseUrl = 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/'
indexes = range(1, 8)
indexStr = ['index.html'] + ['index_{}.html'.format(i) for i in indexes]

pageUrls = [urljoin(baseUrl, link) for link in indexStr]

pageUrls

['http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_1.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_2.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_3.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_4.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_5.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_6.html',
 'http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_7.html']

In [8]:
def createTransacrionsDataFrame():
    df = pd.DataFrame(columns=[
        [
            "全国期货市场", "全国期货市场", "全国期货市场", "全国期货市场",
            "上海期货交易所", "上海期货交易所", "上海期货交易所", "上海期货交易所",
            "上海国际能源交易中心", "上海国际能源交易中心", "上海国际能源交易中心", "上海国际能源交易中心",
            "郑州商品交易所", "郑州商品交易所", "郑州商品交易所", "郑州商品交易所",
            "大连商品交易所", "大连商品交易所", "大连商品交易所", "大连商品交易所",
            "中国金融期货交易所", "中国金融期货交易所", "中国金融期货交易所", "中国金融期货交易所" 
        ], 
        [
            "当月成交量（手）", "当月成交额（亿元）", "当年累计成交量", "当年累计成交额", 
            "当月成交量（手）", "当月成交额（亿元）", "当年累计成交量", "当年累计成交额",
            "成交量", "成交额", "当年累计成交量", "当年累计成交额",
            "成交量", "成交额", "当年累计成交量", "当年累计成交额",
            "成交量", "成交额", "当年累计成交量", "当年累计成交额",
            "成交量", "成交额", "当年累计成交量", "当年累计成交额"
        ]
    ])

    df.index.name = "日期"

    return df

In [9]:
def getDataListFromTransactionDict(trans_dict):
    data_list = []
    default_dict = MonthlyTransation('', 0, 0)

    for index in ['全国期货市场', '上海期货交易所', '上海国际能源交易中心', '郑州商品交易所', '大连商品交易所', '中国金融期货交易所']:

        data_list +=[
            trans_dict.get(index, default_dict).volume,
            trans_dict.get(index, default_dict).turnover,
            trans_dict.get(index, default_dict).cumulativeVolume,
            trans_dict.get(index, default_dict).cumulativeTurnover
        ]
        
    return data_list

In [10]:
df = createTransacrionsDataFrame()

for pageUrl in pageUrls:
    print(pageUrl)
    urls = getCurrentPageUrls(pageUrl)
    
    for url in urls:
        # skip the page returning xls file
        if (url.split('.')[-1] != 'html'):
            continue
        trans_dict = crawlMonthlyTransation(url)
        
        # only for getting date
        sample_value = list(trans_dict.values())[0]
        date = '{}-{}'.format(sample_value.year, sample_value.month)
        
        data_list = getDataListFromTransactionDict(trans_dict)
            
        df.loc[date] = data_list
    

http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index.html
2021/11
全国期货市场 705731971 507884.01 6919065231 5364588.98
上海期货交易所 230806702 171894.0 2200818950 1798136.0
上海国际能源交易中心 7316110 22533.97 66977774 189102.0
郑州商品交易所 227810669 96882.0 2393428467 1001799.0
大连商品交易所 229827521 123085.0 2146230021 1292433.0
中国金融期货交易所 9970969 93489.0 111610019 1083119.0
2021/10
全国期货市场 624779516 478993.35 6213333260 4856704.96
上海期货交易所 186837623 162206.66 1970012248 1626241.63
上海国际能源交易中心 5326030 15518.1 59661664 166568.5
郑州商品交易所 235726661 114119.44 2165617798 904916.94
大连商品交易所 189612676 116223.04 1916402500 1169348.18
中国金融期货交易所 7276526 70926.12 101639050 989629.7
2021/9
全国期货市场 615679950 494494.65 5588553744 4377711.6
上海期货交易所 182751484 153865.23 1783174625 1464034.97
上海国际能源交易中心 5278947 14653.9 54335634 151050.4
郑州商品交易所 215715883 98649.82 1929891137 790797.51
大连商品交易所 200954628 122968.27 1726789824 1053125.14
中国金融期货交易所 10979008 104357.43 94362524 918703.59
2021

2019/7
全国期货市场 415029329 291727.15 2149969492 1577371.83
上海期货交易所 135536203 94537.78 733940303 482613.27
上海国际能源交易中心 3457590 15246.79 23546094 104990.72
郑州商品交易所 138687694 47372.52 663755476 241614.53
大连商品交易所 131385942 72869.81 690701169 362653.47
中国金融期货交易所 5961900 61700.25 38026450 385499.84
http://www.cfachina.org/servicesupport/researchandpublishin/statisticalsdata/monthlytransactiondata/index_2.html
2019/6
全国期货市场 334127019 242277.24 1734940163 1285644.69
上海期货交易所 104544994 70607.67 598404100 388075.49
上海国际能源交易中心 3487004 15059.17 20088504 89743.93
郑州商品交易所 107039287 39246.56 525067782 194242.04
大连商品交易所 113359709 60104.34 559315227 289783.66
中国金融期货交易所 5696025 57259.5 32064550 323799.58
2019/5
全国期货市场 331554683 250212.36 1400813144 1043367.43
上海期货交易所 98804010 64351.82 493859106 317467.82
上海国际能源交易中心 3055918 14906.34 16601500 74684.76
郑州商品交易所 100162833 40314.0 418028495 154995.46
大连商品交易所 122660407 62087.38 445955518 229679.32
中国金融期货交易所 6871515 68552.82 26368525 266540.08
2019/4
全国期货市场 29937277

2016/12
全国期货市场 287686785 174678.98 4137768273 1956316.09
上海期货交易所 113578544 77870.25 1680711841 849774.91
郑州商品交易所 64127915 23875.97 901240809 310297.11
大连商品交易所 107521322 48801.02 1537479768 614052.98
中国金融期货交易所 2459004 24131.74 18335855 182191.1
2016/11
全国期货市场 377694171 216756.22 3850081488 1781637.12
上海期货交易所 150038822 101128.86 1567133297 771904.66
郑州商品交易所 87544500 33925.97 837112894 286421.15
大连商品交易所 138652023 66755.86 1429958446 565251.95
中国金融期货交易所 1458826 14945.54 15876851 158059.36
2016/10
全国期货市场 229452852 114397.43 3472387317 1564880.87
上海期货交易所 84305771 43930.72 1417094475 670775.8
郑州商品交易所 59899825 22760.37 749568394 252495.16
大连商品交易所 84344592 38430.28 1291306423 498496.09
中国金融期货交易所 902664 9276.06 14418025 143113.83
2016/9
全国期货市场 246086898 116083.21 3242934465 1450483.46
上海期货交易所 98626731 48519.42 1332788704 626845.08
郑州商品交易所 59687787 21169.83 689668569 229734.81
大连商品交易所 86689659 35345.82 1206961831 460065.81
中国金融期货交易所 1082721 11048.14 13515361 133837.77
2016/8
全国期货市场 308050228 1503

In [11]:
import xlwings as xw

def getTransactionFromSheet(sheet, year, month):
    row_num, _ = sheet.used_range.shape
    
    while(sheet.range('A' + str(row_num)).value == None):
        row_num -= 1
#     print(sheet.range('A' + str(row_num)).value)
    
    trans_dict = {}
    exchange = ''

    for row in range(3, row_num-1):
        name = sheet.range('A' + str(row)).value
        available_trans = sheet.range('B' + str(row)).value
#         print(available_trans)
        
        if (name != None):
            exchange = name
            transaction = MonthlyTransation(exchange, year, month)
            
        if (''.join(available_trans.split()) == '总额'):
            transaction.setTurnoverVolume(
                sheet.range('I' + str(row)).value,
                sheet.range('C' + str(row)).value
            )
            
            transaction.setCumTurnoverVolume(
                sheet.range('S' + str(row)).value,
                sheet.range('O' + str(row)).value
            )
            
            trans_dict[exchange] = transaction
    
    transaction = MonthlyTransation('全国期货市场', year, month)
    transaction.setTurnoverVolume(
            sheet.range('I' + str(row_num-1)).value,
            sheet.range('C' + str(row_num-1)).value
        )
            
    transaction.setCumTurnoverVolume(
        sheet.range('S' + str(row_num-1)).value,
        sheet.range('O' + str(row_num-1)).value
    )
    
    trans_dict['全国期货市场'] = transaction
            
    return trans_dict

In [12]:
for year in [2014, 2013, 2012]:
    wb = xw.Book('./{}.xls'.format(year))
    total_m = 12
    
    if (year == 2014):
        total_m = 3
        
    for m in list(range(total_m, 0, -1)):
        date = '{}-{}'.format(year, m)
        sheet = wb.sheets['{}月'.format(m)]
        trans_dict = getTransactionFromSheet(sheet, year , m)
    
        df.loc[date] = getDataListFromTransactionDict(trans_dict)
    
    wb.close()

In [14]:
df = df.replace(0, 'NaN')
df

全国期货市场                                                  上海期货交易所  \
                 成交量            成交额       当年累计成交量        当年累计成交额          成交量   
日期                                                                              
2021-11  705731971.0      507884.01  6919065231.0     5364588.98  230806702.0   
2021-10  624779516.0      478993.35  6213333260.0     4856704.96  186837623.0   
2021-9   615679950.0      494494.65  5588553744.0      4377711.6  182751484.0   
2021-8   639171908.0      522229.21  4972873794.0     3883216.96  193734068.0   
2021-7   617433598.0      497658.65  4333701886.0     3360987.74  180346169.0   
...              ...            ...           ...            ...          ...   
2012-5   107600061.0  133429.354469   370672444.0  545390.934774   25163351.0   
2012-4    64091137.0    94884.75919   263072383.0  411961.585305   14267595.0   
2012-3    80974391.0  115926.283461   198981246.0  317076.821115   23051571.0   
2012-2    71324987.0  120639.368413   118006855.0  201150.537498   26611823.0   
2012-1    46681868.0   80511.163741    46681868.0   80511.163741   18117278.0   

                                                   上海国际能源交易中心            ...  \
                   成交额       当年累计成交量       当年累计成交额        成交量       成交额  ...   
日期                                                                       ...   
2021-11  171894.000000  2.200819e+09  1.798136e+06  7316110.0  22533.97  ...   
2021-10  162206.660000  1.970012e+09  1.626242e+06  5326030.0   15518.1  ...   
2021-9   153865.230000  1.783175e+09  1.464035e+06  5278947.0   14653.9  ...   
2021-8   159488.220000  1.600423e+09  1.310170e+06  5936959.0  17321.49  ...   
2021-7   149457.190000  1.406689e+09  1.150682e+06  6231555.0  18821.01  ...   
...                ...           ...           ...        ...       ...  ...   
2012-5    40080.145150  1.072116e+08  1.683453e+05        NaN       NaN  ...   
2012-4    25162.776630  8.204827e+07  1.282651e+05        NaN       NaN  ...   
2012-3    35207.248993  6.778067e+07  1.031024e+05        NaN       NaN  ...   
2012-2    40226.089225  4.472910e+07  6.789512e+04        NaN       NaN  ...   
2012-1    27669.030401  1.811728e+07  2.766903e+04        NaN       NaN  ...   

              郑州商品交易所                   大连商品交易所                               \
              当年累计成交量      当年累计成交额          成交量            成交额       当年累计成交量   
日期                                                                             
2021-11  2.393428e+09  1001799.000  229827521.0  123085.000000  2.146230e+09   
2021-10  2.165618e+09   904916.940  189612676.0  116223.040000  1.916402e+09   
2021-9   1.929891e+09   790797.510  200954628.0  122968.270000  1.726790e+09   
2021-8   1.714175e+09   692147.700  215214483.0  148049.490000  1.525835e+09   
2021-7   1.501352e+09   600716.020  209240462.0  133198.670000  1.310621e+09   
...               ...          ...          ...            ...           ...   
2012-5   8.356498e+07    47830.495   51087774.0   22187.132572  1.461489e+08   
2012-4   5.942345e+07    33512.470   30239357.0   15004.229205  9.506110e+07   
2012-3   4.590181e+07    25537.305   33286786.0   16454.182455  6.482174e+07   
2012-2   2.826463e+07    16110.710   20030002.0   10185.953000  3.153496e+07   
2012-1   1.151802e+07     6651.870   11504953.0    5959.916655  1.150495e+07   

                        中国金融期货交易所                                            
              当年累计成交额         成交量           成交额      当年累计成交量        当年累计成交额  
日期                                                                           
2021-11  1.292433e+06   9970969.0       93489.0  111610019.0      1083119.0  
2021-10  1.169348e+06   7276526.0      70926.12  101639050.0       989629.7  
2021-9   1.053125e+06  10979008.0     104357.43   94362524.0      918703.59  
2021-8   9.301569e+05  11462947.0     105938.34   83383516.0      814346.15  
2021-7   7.821074e+05  11372799.0     108974.66   71920569.0      708407.81  
...          

In [15]:
df.to_excel('./re.xls')

/var/folders/x5/cxp2ts2n7h93w5g276jhwc_00000gn/T/ipykernel_12385/3296885578.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('./re.xls')
